In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/houkongtest/stock_train_data.csv
/kaggle/input/houkongtest/stock_test_data.csv


In [3]:
data = pd.read_csv("/kaggle/input/houkongtest/stock_train_data.csv")

data

,id,date,open,high,low,close,volume,adj_close
0,1,2020-01-01,104.137445,105.336570,101.888063,103.452150,94093,102.546070
1,2,2020-01-02,102.030543,102.243290,100.160189,101.423024,101968,101.260087
2,3,2020-01-03,97.250072,97.450643,96.624196,98.168974,103752,97.330620
3,4,2020-01-04,92.592000,94.713176,89.907974,92.024300,104207,92.640590
4,5,2020-01-05,89.850982,90.289664,88.027048,91.130187,94854,91.772165
...,...,...,...,...,...,...,...,...
1495,1496,2024-02-04,-21.151421,-20.764677,-23.282672,-21.673139,53469,-21.670244
1496,1497,2024-02-05,-18.697602,-18.685963,-20.843889,-18.447026,51435,-18.495587
1497,1498,2024-02-06,-14.312172,-13.467672,-17.200586,-12.386975,58108,-12.287926
1498,1499,2024-02-07,-9.496567,-7.847834,-9.859621,-11.494308,48258,-11.392723


In [4]:
data = data.drop(columns = ['id', 'date', 'adj_close'])

data

,open,high,low,close,volume
0,104.137445,105.336570,101.888063,103.452150,94093
1,102.030543,102.243290,100.160189,101.423024,101968
2,97.250072,97.450643,96.624196,98.168974,103752
3,92.592000,94.713176,89.907974,92.024300,104207
4,89.850982,90.289664,88.027048,91.130187,94854
...,...,...,...,...,...
1495,-21.151421,-20.764677,-23.282672,-21.673139,53469
1496,-18.697602,-18.685963,-20.843889,-18.447026,51435
1497,-14.312172,-13.467672,-17.200586,-12.386975,58108
1498,-9.496567,-7.847834,-9.859621,-11.494308,48258


In [5]:
X = data.drop(columns = ['close'])
y = data['close']

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
regressor = RandomForestRegressor(n_estimators=200,criterion='squared_error',max_depth=50)
regressor.fit(X, y)

RandomForestRegressor(max_depth=50, n_estimators=200)

In [ ]:
'''
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

def random_forest_regressor_cross_validation(X, y, n_splits=5, n_estimators=200, max_depth=50, random_state=None):
    """
    Performs k-fold cross-validation for a Random Forest Regressor.

    Args:
        X (numpy.ndarray): The feature matrix.
        y (numpy.ndarray): The target vector.
        n_splits (int): The number of folds for cross-validation. Default is 5.
        n_estimators (int): The number of trees in the Random Forest. Default is 200.
        max_depth (int): The maximum depth of the trees in the Random Forest. Default is 50.
        random_state (int or None): Random seed for reproducibility. Default is None.

    Returns:
        tuple: A tuple containing:
            - mean_rmse (float): The mean RMSE across all folds.
            - std_rmse (float): The standard deviation of the RMSE across all folds.
            - mean_r2 (float): The mean R-squared across all folds.
            - std_r2 (float): The standard deviation of the R-squared across all folds.
            - all_rmse (list): A list of RMSE for each fold.
            - all_r2 (list): A list of R-squared for each fold.
    """

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    all_rmse = []
    all_r2 = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Initialize and train the Random Forest Regressor model
        regressor = RandomForestRegressor(n_estimators=n_estimators,
                                            criterion='squared_error',
                                            max_depth=max_depth,
                                            random_state=random_state)
        regressor.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred = regressor.predict(X_val)

        # Calculate the metrics
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        r2 = r2_score(y_val, y_pred)

        all_rmse.append(rmse)
        all_r2.append(r2)

    # Calculate the mean and standard deviation of the metrics
    mean_rmse = np.mean(all_rmse)
    std_rmse = np.std(all_rmse)
    mean_r2 = np.mean(all_r2)
    std_r2 = np.std(all_r2)

    return mean_rmse, std_rmse, mean_r2, std_r2, all_rmse, all_r2

if __name__ == '__main__':
    # Generate some dummy data for demonstration
    X = np.random.rand(100, 10)  # 100 samples, 10 features
    y = np.random.rand(100)  # Regression target

    # Perform cross-validation
    mean_rmse, std_rmse, mean_r2, std_r2, all_rmse, all_r2 = random_forest_regressor_cross_validation(X, y)

    # Print the results
    print("All RMSE:", all_rmse)
    print("Mean RMSE:", mean_rmse)
    print("Standard deviation of RMSE:", std_rmse)
    print("All R-squared:", all_r2)
    print("Mean R-squared:", mean_r2)
    print("Standard deviation of R-squared:", std_r2)
'''

In [16]:
test = pd.read_csv("/kaggle/input/houkongtest/stock_test_data.csv")
test

,date,open,high,low,volume,adj_close
0,2024-02-09,64.217540,67.120519,62.056126,109241,63.473761
1,2024-02-10,63.996529,66.075604,63.633681,105939,65.039274
2,2024-02-11,62.983233,64.875152,61.678532,105692,65.083193
3,2024-02-12,63.023560,64.535805,61.275462,99532,64.323752
4,2024-02-13,64.880834,66.720266,64.708353,103976,66.364234
...,...,...,...,...,...,...
495,2025-06-18,29.048598,30.268272,28.232786,144479,28.705003
496,2025-06-19,30.670911,32.326131,27.826621,143250,28.947784
497,2025-06-20,29.426022,31.409296,27.348696,135449,28.192249
498,2025-06-21,26.292295,27.369338,26.042563,139356,24.342610


In [18]:
test = test.drop(columns = ['date', 'adj_close'])

In [19]:
test

,open,high,low,volume
0,64.217540,67.120519,62.056126,109241
1,63.996529,66.075604,63.633681,105939
2,62.983233,64.875152,61.678532,105692
3,63.023560,64.535805,61.275462,99532
4,64.880834,66.720266,64.708353,103976
...,...,...,...,...
495,29.048598,30.268272,28.232786,144479
496,30.670911,32.326131,27.826621,143250
497,29.426022,31.409296,27.348696,135449
498,26.292295,27.369338,26.042563,139356


In [26]:
pred = regressor.predict(test)

In [27]:
pred

array([ 65.88309338,  66.39172059,  64.34817876,  64.27453688,
        66.7101747 ,  70.36956468,  70.57373435,  70.45088131,
        70.20280803,  66.90309061,  66.8810902 ,  62.76358052,
        64.58509532,  66.22001869,  63.78849072,  66.76548866,
        71.41542207,  70.86867245,  69.32544716,  69.61247925,
        73.1914825 ,  70.60990394,  68.32308652,  63.64059269,
        63.46348952,  67.23837757,  63.80506713,  65.54680253,
        66.62161284,  62.66430211,  57.23056588,  63.17079823,
        65.85640183,  69.60562517,  69.76692102,  65.65543648,
        58.83584422,  64.11536866,  59.07945918,  64.49636809,
        63.66891801,  63.32322478,  57.65519703,  59.23329149,
        57.62372536,  56.4323362 ,  58.62047726,  61.39506015,
        58.69822838,  56.45597314,  58.65229877,  64.10846787,
        63.14264598,  65.43847498,  68.09600378,  68.75191458,
        70.83602125,  73.05725683,  76.68907463,  81.15116968,
        80.87735577,  81.49788409,  83.8624678 ,  85.13

In [29]:
submission = pd.DataFrame({
    "id": range(1501,2001),  # 自动生成 ID
    "Predicted": pred
})
submission.to_csv("submission.csv", index=False)

In [30]:
submission

,id,Predicted
0,1501,65.883093
1,1502,66.391721
2,1503,64.348179
3,1504,64.274537
4,1505,66.710175
...,...,...
495,1996,30.300995
496,1997,29.885527
497,1998,29.428796
498,1999,26.770793
